# Набор инструментов


Тренировка нейросетей может занимать значительное время, так что есть несколько важных вещей, которые надо не забыть:

- сохранять промежуточные/лучшие веса (делать чекпоинты)
- писать логи в адекватном формате (так, чтобы они не терялись при перезапуске тетрадки)


**План тетрадки**
1. Логгирование и tensorboard
2. Сохранение-восстановление весов
3. Dataset API в pytorch
4. Выбор LR
5. Практика
6. Затухающие градиенты и Residual-блоки


Есть множество вариантов хранения логов и артефактов (весов) обучения. Прежде чем выбрать какое-то коробочное решение имеет смысл попробовать собрать несколько своих велосипедов.


## Логгирование

В предыдущих тетрадках мы хранили тренировочные логи в словарях со списками. Это неплохой вариант, его можно сериализовать в файлы и рисовать в отдельном коде. Однако эти логи бесполезны без знания _схемы_.

Довольно распространенным вариантом является формат tensorboard: это бинарный формат (не текстовые, как json) с готовым просмотрщиком.




[Tensorboard](https://www.tensorflow.org/tensorboard) -- это pip-installable web-приложение.

```
tensorboard --logdirs=./some-folder/with/events-files
# зайти на http://localhost:6006
```
<img src="./img/tb.png"/>

При желании, можно написать python-код для парсинга логов и делать что-то с ними руками.

Чтобы писать логи в pytorch есть класс `torch.utils.tensorboard.SummaryWriter`

In [ ]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter("./check-this/")

fake_loss = 1 / np.arange(1, 100)
for global_step, point in enumerate(fake_loss):
    writer.add_scalar("lossy", point, global_step=global_step)
writer.close()

In [ ]:
! ls check-this

# поставьте tensorboard
# запустите его в папке с логами

TB умеет рисовать множество графиков, для этого достаточно писать их в разные папки, лежащие в `--logdir`.

Хорошая практика: для каждого запуска обучения заводить отдельную папку с логами, записывать туда словарь с гиперпараметрами и логи.

## Состояния модели и оптимизатора

Нам часто бывает необходимо сохранить/загрузить веса модели. Расхожее название для этого `checkpoint`.
В DL термином `checkpointing` называют так же метод бекпропа, позволяющий экономить память ценой дополнительных вычислений (https://pytorch.org/docs/stable/checkpoint.html#torch-utils-checkpoint).

У торчевых моделей и оптимизаторов есть методы для получения и загрузки состояний:
- `.state_dict()` возвращает словарь (или почти словарь) с весами
- `.load_state_dict()` загружает веса из словаря в модельку



Для сохранения/загрузки словарей с тензорами в файлы есть простые функции `torch.save(some_dict, path)`, `torch.load(path)`. Сравните с использованием `pickle` или `json`!


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


some_model = nn.Sequential(nn.Linear(10, 10))
print(some_model.state_dict())

opt = optim.Adam(some_model.parameters())
print(opt.state_dict())


torch.save({"model_stuff": some_model.state_dict(), "opt_stuff": opt.state_dict()}, "./that.is.it")

In [ ]:
torch.load("./that.is.it")

## Dataset API

Подготовка данных легко может стать бутылочным горлышком, когда на подготовку очередного батча уходит больше времени, чем на forward+backward проходы по сети.
Проблема усложняется особенностями python: чтобы использовать несколько ядер CPU для подготовки данных надо постараться.

В Pytorch работа с данными строится на двух классах из [troch.utils.data](https://pytorch.org/docs/stable/data.html): `Dataset` и `DataLoader`:

- `Dataset` отвечает за подготовку одного примера
- `DataLoader` отвечает за выбор примеров, склейку их в один батч и распараллеливание на CPU, поддерживает итерирование.


Для решения задачи обычно пишут кастомные Dataset-классы, для этого нужно написать всего две функции:
- `.__len__(self)` возвращает количество примеров в датасете;
- `.__getitem__(self, item)` возвращает item-ный по счету пример из датасета.

Задачи DataLoader достаточно сложно аккуратно реализовать и лучше использовать готовый. Он довольно гибкий, все основные моменты кастомизируются заданием функций:
```
torch.utils.data.DataLoader(
    dataset,            # собственно экземпляр класса Dataset, из которого надо доставать примеры
    batch_size=1,       # количество примеров в батче
    drop_last=False,    # нужно ли при итерировании выбрасывать неполные батчи? (такое бывает, если число примеров не делится нацело на batch_size
    shuffle=False,      # перемешивать ли примеры
    sampler=None,       # чтобы перемешивать примеры кастомно
    batch_sampler=None, # чтобы использовать кастомный отбор примеров в батч
    num_workers=0,      # на сколько процессов запараллелить подготовку данных
    collate_fn=None,    # функция, которая будет склеивать примеры в батчи
    # остальные аргументы более технические, сейчас можно не рассматривать
    pin_memory=False,   
    timeout=0, 
    worker_init_fn=None, 
    multiprocessing_context=None, 
    generator=None)
```

Напишите два датасета для работы с FashionMnist: один готовит данные как вектора, другой как картинки


**NB: FashionMNIST возвращает картинки в формате PIL.Image.Image, чтобы сделать из него понятный np.array, просто вызовите np.array(PIL_IMAGE)**

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST


class VectorSet:
    def __init__(self, train=True):
        self.data = FashionMNIST("./tmp", train=train, download=True)
    
    def __len__(self):
        <your code>
    
    def __getitem__(self, item):
        # сделайте вектор с float32 числами
        <your code>
        return dict(
            sample=...,
            label=...,
        )

vs = VectorSet()
print(vs[0])
        
class ImageSet:
    def __init__(self, train=True):
        self.data = FashionMNIST("./tmp", train=train, download=True)
    
    def __len__(self):
        <your code>
    
    def __getitem__(self, item):
        # сделайте одноканальную картинку [1, 28, 28] с float32
        <your code>
        return dict(
            sample=...,
            label=...,
        )
    
ms = ImageSet()
print(ms[0])

In [ ]:
# проверьте итерирование, именно его мы используем в train-loop'е
vl = DataLoader(vs, batch_size=4)
for batch in vl:
    for k, v in batch.items():
        print(k, v.shape)
    raise

## Замечания по Dataset/Dataloader

1. Dataset может возвращать что угодно (туплы или словари) с отдельными числами или массивами (numpy или уже готовыми torch.tensor). Удобно возвращать словари с читабельными ключами. Тогда 

2. Имеет смысл поглядеть в [стандартный collate_fn](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/_utils/collate.py#L42): он умеет клеить в батчи и конвертировать в тензора самые разнообразные данные. Однако, массивы разной длины не сможет. Так что для 



## Выбор оптимального LR


Для выбора оптимального LR удобно использовать т.н. Learning Rate Range Test, часто процедуру называют просто find_lr. Под капотом проход по тренировочной эпохе с lr, изменяемым на каждом батче по формуле:

$$
\mathrm{it} = \frac{\mathrm{step}}{\mathrm{total steps}}\\
\mathrm{lr} = \exp\left\{ 
    (1 - t ) \log a + t \log b
\right\}
$$

Чтобы поменять LR для всех оптимизируемых параметров, можно пройтись по ним циклом:

```
for param_group in optimizer.param_groups:
    param_group['lr'] = lr
```


<img src="https://www.jeremyjordan.me/content/images/2018/02/lr_finder.png"/>

_картинка из бложика [Jeremy Jordan](https://www.jeremyjordan.me/nn-learning-rate/)_


Идея приема простая: пока LR меньше некоторого порога на каждом шаге градиентного спуска веса просто не меняются (в частности из-за особенностей операций с плавающей точкой).
При очень большом LR мы шагаем слишком далеко и уходим от точки экстремума. 

Оптимальный LR лежит где-то между ними. Экспоненциальная формула изменения LR позволяет с должным качеством найти хорошую точку.



Если интересно: [статья , в которой эту технику предложили и активно использовали](https://arxiv.org/pdf/1506.01186.pdf).


**Some math notes**

У типов данных с плавающей точкой есть арифметические особенности:

$$
x + \delta == x,\,\mathrm{если}\; \delta < 5.96 \cdot 10^{-8} x
$$

К слову, это еще одна причина присматривать за величинами активаций, нормировать данные и таргет в случае регрессии. Можно было бы перейти на float64, но (вычислительно) дешевле быть аккуратными на float32.

Благодаря NVIDIA на новом железе получается эффективно пользоваться форматом fp16, но это требует некоторых трюков.

In [ ]:
# просто попробуйте
w = np.float32(1.0)
too_much = np.float32(5.97e-8)
too_not = np.float32(5.96e-8)
print(w == w + too_much)
print(w == w + too_not)

## Practice!


1. Пишем сеть (полносвязную), лосс
2. Задаем Dataset, и думаем про transform
3. Пишем train-loop & find-lr

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import FashionMNIST

# сделайте DataLoader выдающий вектора из FashionMNIST

train_loader = ...
val_loader = ...


# проверочный батч, пригодится дальше
for some_batch in train_loader:
    print(some_batch)
    break

Сделайте простую сеть для классификации FashionMNIST с тремя полносвязными слоями с нелинейностями между ними.
Все промежуточные тензора пусть имеют 40 каналов.

In [ ]:
class VeryModel(nn.Module):
    def __init__(self):
        super().__init__()
        pass
    
    def forward(self, x):
        pass
    
    def compute_all(self, batch):  # удобно сделать функцию, в которой вычисляется лосс по пришедшему батчу
        loss = ...
        metrics = dict()
        return loss, metrics

# проверяйте работоспособность сразу
net = VeryModel()
net.compute_all(some_batch)

Напишите find_lr (для подбора хорошего LR) и train_model (для тренировки и валидации).

Тренировочные кривые можно рисовать здесь или сохранять с помощью SummaryWriter для tensorboard.
(результаты find_lr лучше рисовать здесь, будет проще подобрать масштаб)

In [ ]:
def find_lr(model, opt, trainloader):
    pass

def train_model(model, opt, trainloader, valloader, epochs=10):
    pass

In [ ]:
# подберите LR
net = VeryModel()
opt = optim.SGD(net.parameters(), lr=1e-3)
find_lr(net, opt, train_loader)

In [ ]:
net = VeryModel()
opt = optim.SGD(net.parameters(), lr=<...>)
train_model(net, opt, train_loader, val_loader)

## Эксперимент с затухающими градиентами

Сделайте функцию, собирающую нормы градиентов по слоям.
Функция должна работать с GPU и CPU-моделями!

Ключи в словаре с нормами сделайте по именам слоев (используйте `model.named_parameters()` из предыдущей тетрадки).

In [ ]:
def get_grad_norms(model):
    <your code>
    return {"some.weight": some float}

# код для проверки корректности
model = nn.Sequential(
    nn.Linear(7, 11),
    nn.Sigmoid(),
    nn.Linear(11, 10),
)

x = torch.ones(13, 7)
loss = model(x).mean()
loss.backward()

assert get_grad_norms(model).keys() == {"0.weight", "0.bias", "2.weight", "2.bias"}

if torch.cuda.is_available():
    device = "cuda"
    model.to(device)
    x = x.to(device)
    loss = model(x).mean()
    loss.backward()
    assert get_grad_norms(model).keys() == {"0.weight", "0.bias", "2.weight", "2.bias"}
    print("All is fine")
else:
    print("GPU unchecked")

### Глубокая сеть

Сделайте глубокую сеть из полносвязных слоев и нелинейностей.
8 FC-слоев с нелинейностями между ними, постройте нормы градиентов по слоям от номера шага.
Размеры промежуточных тензоров возьмите 20 или 40 каналов.

Добавьте в функцию `train_model` вывод норм градиентов.

Ожидаемая картинка выглядит примерно так: <img src="./img/sad_network.png/">

In [ ]:
net = VeryModel()
opt = optim.SGD(net.parameters(), lr=<...>)
train_model(net, opt, train_loader, val_loader)

Есть по крайней мере несколько способов, которые могут помочь заставить эту сеть учиться (без изменения глубины):
- использовать другую функцию активации
- выбрать LR для каждого слоя
- собрать сеть из других блоков

Вы можете попробовать первый или второй (или какой-нибудь свой) вариант, но сопроводите, пожалуйста, код письменными рассуждениями.

### Residual Blocks

Если сеть собирать из блоков вида
```
y = x + F(x)
```
где F(x) -- это FC-слой + нелинейность, градиенты будут протекать гораздо лучше (поскольку есть путь без нелинейностей).

<img src="./img/resblock.png">

Реализуйте каким-либо образом ResNet с 8 нелинейностями с тем же количеством каналов.

Постройте картинку с градиентами по слоям для каждого шага.

In [ ]:
net = ResModel()
opt = optim.SGD(net.parameters(), lr=<...>)
train_model(net, opt, train_loader, val_loader)